In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: Depression

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Depression&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Depression

In [4]:
paper_link_list = []
#80
for i in range(80):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Depression&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Depression"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

Prevalence and Incidence of Depression during Interferon-Based Antiviral Therapy in Chronic Hepatitis C Patients in the Republic of Korea
Original Article : Anxiety, Depression and Quality of Life in Patients with Irritable Bowel Syndrome
Original article : Prevalence of Depression and Anxiety in Outpatients with Chronic Airway Lung Disease
ORIGINAL ARTICLE : Relationships between Metabolic Syndrome Component and Depression, Stress
The effect of depression and health-related quality of life on the outcome of hemodialysis patients
Moderating Effect of Family Support on the Relationship between Parenting Stress on Depression of Immigrant Women
ORiginal Article : Factors Associated with Anxiety and Depression in Korean Patients with Inactive Inflammatory Bowel Disease
Korean Housewives Living through Depression: A Grounded Theory Study
The Effects of Herb Aroma Components on the Reduction of Depression -Focused on University Students-
Original Article : Strain Differences in the Chronic M

웃음치료가 염증성 장질환 환자의 불안 및 우울에 미치는 효과
16주간의 복합운동이 고령여성의 치매관련인자, 우울감 및인지기능에 미치는 영향
항암화학요법을 받는 노인의 증상경험과 우울이 낙상 효능감에 미치는 영향
남녀 아동의 또래 괴롭힘의 가해와 피해에 또래관계, 우울 및 공격성이 미치는 영향
지역사회 만성질환노인의 약물사용과 우울예방 교육 프로그램 효과
노인의 영적안녕, 영적요구 및 우울 간의 관계
여성노인의 세대 간 사회적 지원교환과 자기효능감이우울정도에 미치는 영향
단기 생애회고요법이 말기 암 환자의 영적 안녕, 우울 및 불안에 미치는 효과
집단재활체조 프로그램이 뇌졸중 환자의 우울과 재활동기에 미치는 효과
성인여성의 도시 숲 이용행태에 따른 삶의 만족과 우울의 차이
180
찾아가는 인지기능향상프로그램이 후기 노인의 고독감, 우울 및 자아존중감에 미치는 효과
여성중심의 재발예방 프로그램이 여성알코올중독자의 금주 자기효능감과 우울에 미치는 효과
임금근로자의 우울관련요인: 주관적 건강과 스트레스 인지를 중심으로
크론병 환자의 우울과 위험요인
재활자기관리 노트를 활용한 간호중재 프로그램이 우울, 재활동기, 자기효능감에 미치는 효과
한국 청소년의 수면시간, 스트레스 및 우울증이 구강건강 상태에 미치는 영향
웃음요법이 유방암 생존자의 우울, 삶의 질, 극복력, 면역반응에 미치는 효과
S구 중도장애인의 자살생각 관련 변인 연구 : 일상생활활동, 자살시도경험, 우울변수를 중심으로
치매간병가족을 위한 행복 더하기 프로그램이 가족간병자의 수면, 우울, 간병 부담감, 삶의 질에 미치는 효과
역량강화 프로그램이 저소득층 여성독거노인의 자아존중감과 우울에 미치는 효과
190
이혼가정아동의 우울: 가족유형별 비교를 중심으로
발반사 마사지가 골관절염 재가 여성노인의 관절통증, 관절강직, 우울 및 수면에 미치는 효과
아동 청소년기 우울증의 예방 개입에 대한 고찰
대학생이 지각하는 스트레스와 우울 및 자아존중감
연구논문 : 아로마를 이용한 두피, 손 마사지가 폐경기 여성의 우울 및 심

가미소요산(加味逍遙散)의 항산화 효과와 serotonin 대사과정에 미치는 영향
청소년의 휴대 전화 사용 실태 및 정신과적 문제와의 연관성
세월호 참사로 자녀를 상실한 부모의 삶의 질에 영향을 미치는 요인
간호대학생의 스마트폰 중독에서의 자기효능감과 자기통제력의 매개효과
혈액투석 환자의 증상 클러스터와 삶의 질
시설보호아동과 일반가정아동의 정신건강 수준에 관한 비교
Childhood Domestic Violence, Alcoholic Parents, and Adult Psychological Features
소방공무원의 외상 후 스트레스 모형구축
의과 대학생의 신체부위 불만족과 성격특성에 대한 연구
듀센·베커 근이영양증 아동 부모의 우울과 삶의 질
390
혈액투석 환자의 수면의 질에 영향을 주는 신체적, 심리적, 생리적 요인
성인 남성의 건강 관련 삶의 질에 영향을 미치는 요인에 관한 경로 분석
임상간호사의 사회심리적 건강과 건강지각, 건강관련 행위
농촌 노인과 도시 노인의 참여에 미치는 영향요인 비교
이혼가정 아동의 학교적응: 아동의 개인적 변인과 가족 변인을 중심으로
산전후 우울 변화-성장혼합모형을 이용한 단기종단연구
성별에 따른 대학생의 일상적 스트레스, 사회적 지지, 속박감 및 정신건강의 관계
원저 : 주사 환자의 삶의 질과 정신건강
증례 : 청신경초종 수술 이후 어지럼증에 대한 침구 치료 1례
뇌졸중 환자의 운전수행에 영향을 미치는 요인
400
노인 불면에 대한 EFT 불면 치료 프로그램(EFT-1)의 효과 평가를 위한 예비적 연구
Original Article : Impact of Psoriasis on Quality of Life: Relationship between Clinical Response to Therapy and Change in Health-related Quality of Life
도시 빈곤 노인의 낙상발생 위험요인에 관한 연구
지역사회 거주 노인의 금연의도 영향요인
자살 위험군 청소년을 위한 소그룹 작업치료 프로그램의 효과 : 

뇌졸중 환자의 우울감 감소를 위한 스트레스 면역훈련을 기반으로 한 원예치료프로그램
예비노인과 노인의 삶의 만족에 관한 연구
취업여성의 우울증상 예측요인: 한국의 미혼, 기혼 취업여성 비교
뇌졸중 환자의 삶의 질 구조모형
우울을 동반하거나 동반하지 않은 강박증상군과 우울증상군의 변별 : 자의식, 예방초점 및 자기비난을 중심으로
심박변이도 지표에 나타난 자율신경 상태와 우울, 불안 및 분노 설문검사 척도 간의 상관성 평가
연령별 체질량지수 3백분위수 미만을 희망하는 여고생의 식습관과 심리적 상태
대학생의 데이트폭력 가해 예측 요인
발달협응장애 동반 유무에 따른 주의력결핍 과잉행동장애 아동들의 임상적 특성
한국 청소년 대상 성격양식 척도 PSI-Ⅱ의 타당화 연구
580
RESEARCH : Population genetic structure analysis and effect of inbreeding on body weights at different ages in Iranian Mehraban sheep
자살생각과 관련된 인지적 요인 : 내부초점적 반응양식의 매개효과와 탈중심화의 조절효과 중심으로
에코세대의 취업변화와 자살생각
지역사회 상황적 특성이 지역사회 간 정신건강 차이에 미치는 영향
적작약(赤芍藥)이 강제수영부하시험에서 CRF, c-Fos 와 TH에 미치는 영향
청소년 정신건강증진을 위한 이야기치료 프로그램의 효과
Original article : The Involvement of AMPA Receptor in the Antidepressant-like Effects of the Portulaca Oleracea L. Extract in Mice
외상 후 스트레스 장애 아동의 우울·불안 및 자기효능감 미술치료 단일사례연구
국제결혼한 한국남성의 정신건강과 결혼행복에 대한 연구 : 부부갈등과 의사소통의 매개효과를 중심으로
Effects of Docosahexaenoic Acid on Neurotransmission
590
학업 목적 외의 인터넷 사용과 청소년들의

가정외보호아동의 사회적 자본과 문화적 자본이 학업성취에 미치는 영향
원저 : 직장 카르시노이드 종양에서 Ki-67 발현의 예후적 중요성
맞춤형 방문건강관리사업 노인대상자의 고혈압군, 당뇨군, 고혈압·당뇨군별 건강상태 및 건강행태 비교
슬관절염에 대한 뜸 치료의 유효성 및 안전성 연구: 무작위 대조 예비 임상연구 프로토콜
청소년정서행동발달검사 표준화연구 자료를 활용한 학교폭력 피해 전국유병률 및 관련요인 조사
치매 노인에게 적용한 작업 중심 회상치료와 의사소통 중심에 따른 회상치료의 효과
원저 : 경골 고평부 골절에서 관절경을 이용한 관절강 내 정복술 및 내고정술
산림치유 이용자의 자연연관성, 감정, 각성상태가 스트레스 회복에 미치는 영향 -50대 여성을 중심으로-
대학병원 응급실을 내원한 자살 시도자들의 특성
응급실 내원 노인정신과 환자의 특징
760
지하철 기관사의 외상후스트레스장애 증상에 대한 중재효과
아르데코(Art Deco) 도자디자인의 조형적 특성
원저 : 닭에서 저병원성 조류인플루엔자와 뉴캐슬 바이러스의 복합감염에 따른 임상적, 병리학적 연구
숲체험 활동이 초등학생의 숲에 대한 태도 및 정신건강에 미치는 효과
작업치료사의 직무 스트레스와 소진
심장이식 수혜자의 삶의 질 관련 요인에 대한 메타분석
감압 튀김 방법이 감자튀김의 품질에 미치는 영향
한국 근로자의 요통 유병률과 근로환경의 연관성에 관한 연구
미국 이주 한인노인의 정신건강에 영향을 미치는 요인에 관한 탐색 : 인구사회학적 특성과 사회자본 특성을 중심으로
세대통합 원예활동 프로그램이 성별 노인의 삶의 질 및 아동의 사회심리적 행동 특성에 미치는 영향
770
숲 걷기 명상이 중년 여성의 감정 상태 및 자기자각에 미치는 영향
노인과 아동 간 세대통합 원예활동 프로그램이 노인의 삶의 질 및 아동의 사회심리적 행동 특성에 미치는 영향
논문 : 산림치유 효과 측정 지표로써 과산화지질의 활용가능성 분석
직장 내 물리적, 화학적 유해인자 노출과 정신건강 문제의 관련성
Monoamine Oxidase and

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a3289a1475c5992d7f7a54760bb41745',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c901ff133a205c0cffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=967e0980805dcc62ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=db5d7dbd46247554e9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9f95a90c7302c9b7ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ba18aca53c00df77ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7d4941f01211ebfd7ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2e8f0758562ec28e6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c585df70f273ab3fffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

793

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 793/793 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a3289a1475c5992d7f7a54760bb41745',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c901ff133a205c0cffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=967e0980805dcc62ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=db5d7dbd46247554e9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9f95a90c7302c9b7ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ba18aca53c00df77ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7d4941f01211ebfd7ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2e8f0758562ec28e6aae8a972f9116fb',
 'http://www.ris

In [9]:
len(paper_url)

793

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./4_Depression.xlsx')
    
print("Running Time : ", time.time() - start)

0 ERROR
1 ERROR
2 ERROR
3 ERROR
4 paper clear
5 ERROR
6 ERROR
7 ERROR
8 ERROR
9 ERROR
10 ERROR
11 ERROR
12 paper clear
13 ERROR
14 ERROR
15 ERROR
16 ERROR
17 ERROR
18 ERROR
19 ERROR
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 ERROR
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 paper clear
45 ERROR
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 ERROR
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 paper clear
68 paper clear
69 paper clear
70 ERROR
71 paper clear
72 paper clear
73 paper clear
74 paper clear
75 paper clear
76

552 paper clear
553 paper clear
554 paper clear
555 paper clear
556 paper clear
557 paper clear
558 paper clear
559 ERROR
560 paper clear
561 paper clear
562 paper clear
563 ERROR
564 paper clear
565 ERROR
566 ERROR
567 ERROR
568 ERROR
569 paper clear
570 ERROR
571 paper clear
572 ERROR
573 paper clear
574 paper clear
575 paper clear
576 ERROR
577 paper clear
578 paper clear
579 paper clear
580 paper clear
581 paper clear
582 ERROR
583 paper clear
584 ERROR
585 paper clear
586 ERROR
587 paper clear
588 ERROR
589 paper clear
590 paper clear
591 ERROR
592 ERROR
593 ERROR
594 ERROR
595 ERROR
596 paper clear
597 paper clear
598 paper clear
599 ERROR
600 ERROR
601 paper clear
602 ERROR
603 ERROR
604 ERROR
605 ERROR
606 ERROR
607 ERROR
608 ERROR
609 ERROR
610 ERROR
611 paper clear
612 ERROR
613 ERROR
614 paper clear
615 paper clear
616 paper clear
617 paper clear
618 ERROR
619 paper clear
620 ERROR
621 paper clear
622 ERROR
623 ERROR
624 paper clear
625 paper clear
626 paper clear
627 paper 